***GENERATED CODE FOR denailqueryedit PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
from pyspark.ml.feature import StandardScaler
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer
import json
from pyspark.sql.functions import col, round
from pyspark.sql.functions import col, udf
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, when
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def vectorAssemblerTransform(df, param):

    dfReturn = df

    if (type(param) == str):
        outcol = param + "_vector"
        assembler = VectorAssembler(inputCols=[param], outputCol=outcol)
        dfReturn = assembler.transform(dfReturn)
        return dfReturn

    if (type(param) == list):
        vecAssembler = VectorAssembler(inputCols=param, outputCol="features")
        new_df = vecAssembler.transform(df)
        return new_df


def to_array(col):
    def to_array_(v):
        return v.toArray().tolist()
    return udf(to_array_, ArrayType(DoubleType()))(col)


def standardScalarTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: '0.0'})
    scalarFlags = transformationData["std_scalar"]
    if scalarFlags["mean_flag"]:
        stdflag = False
        meanflag = True
    elif scalarFlags["std_flag"]:
        stdflag = True
        meanflag = False
    outcol = feature + "_standardscalar"
    featureVector = feature + "_vector"
    dfReturn = vectorAssemblerTransform(dfReturn, feature)

    standardscale = StandardScaler(inputCol=featureVector, outputCol=outcol, withStd=stdflag,
                                   withMean=meanflag)
    scaledata = standardscale.fit(dfReturn).transform(dfReturn)
    dfReturn = scaledata.withColumn("final_col", to_array(scaledata[outcol]))\
        .select(scaledata.schema.names + [col("final_col")[0]])

    dfReturn = dfReturn.drop(outcol).drop(featureVector)\
        .withColumnRenamed("final_col[0]", outcol)
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[outcol], 2))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfEntry', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
                                                'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'DateOfEntry'}, {'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DateOfEntry')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTModifier', 'transformation_label': 'String Indexer'}], 'feature': 'CPTModifier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '2399', 'mean': '31.05', 'stddev': '13.18', 'min': '00', 'max': 'XU', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTModifier'}, {'feature_label': 'CPTModifier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTModifier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PracticeCategory', 'transformation_label': 'String Indexer'}], 'feature': 'PracticeCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '2399', 'mean': '', 'stddev': '', 'min': 'Charles Lee Ginsburgh', 'max': 'TPMG', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PracticeCategory'}, {'feature_label': 'PracticeCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PracticeCategory')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateOfService', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfService', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'DateOfService'}, {'feature_label': 'DateOfService', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DateOfService')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCategory', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '2399', 'mean': '119527.27', 'stddev': '512384.2', 'min': '00731', 'max': 'Q4170', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCategory'}, {'feature_label': 'CPTCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '2399', 'mean': '', 'stddev': '', 'min': '<Unspecified>', 'max': 'X-Ray', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = standardScalarTransform(transformationDF, {'transformationsData': [{'feature_label': 'DiagnosisCodeID1', 'std_scalar': {'mean_flag': 'True', 'std_flag': 'False'}, 'transformation_label': 'Standard Scalar'}], 'feature': 'DiagnosisCodeID1', 'transformation': [{'transformation': 'Standard Scalar', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'min', 'selected': 'True', 'stats': {
            'count': '2399', 'mean': '41839.47', 'stddev': '23100.1', 'min': '13571', 'max': '94869', 'missing': '0'}, 'updatedLabel': 'DiagnosisCodeID1'}, {'feature_label': 'DiagnosisCodeID1', 'std_scalar': {'mean_flag': 'True', 'std_flag': 'False'}, 'transformation_label': 'Standard Scalar'})
        transformationDF = transformationDF.drop('DiagnosisCodeID1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '2399', 'mean': '', 'stddev': '', 'min': 'After Hours Care Newport News', 'max': 'Yorktown Family Medicine', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityCategory'}, {'feature_label': 'FacilityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityCategory')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run denailqueryeditHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'url': '/FileStore/platform/sampleData/5f7c30ba329c0a1106d45bfc/part-00000-tid-4571400751018531844-3f7083bc-7697-4ca2-962c-4bf4d6cbfdd8-13871-1-c000.csv', 'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'NextGen_Tidewater_Physicians', 'optionalDB': 'NextGen_Tidewater_Physicians', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT TOP(100000) cd.Amount, cd.DateOfEntry,cd.PrimaryInsuranceBal,cd.CPTModifier,Pr.PracticeCategory,cd.DateOfService, cp.CPTCategory,dep.DepartmentCategory, cd.DiagnosisCodeID1, fac.FacilityCategory, status=(SELECT CASE trc.ReasonType WHEN 'Denial' THEN 1 ELSE 0 END) FROM chargedetail cd inner join TransactionReason as tr on tr.ChargeDetailID = cd.ChargeDetailID inner join TransactionReasonCode as trc on tr.TransactionReasonCodeID = trc.TransactionReasonCodeID inner join Practice as Pr on Pr.PracticeID = cd.PracticeID inner join CptCode as cp on cp.CPTCodeID = cd.CPTCodeID inner join Department as dep on dep.DepartmentID = cd.DepartmentID inner join Facility as fac on fac.FacilityID = cd.FacilityID where cd.DateOfEntry between '1/1/2019' and '1/30/2019' and cd.Amount > .01 and exists ( select distinct ChargeDetailID as cd from TransactionReasonCode as tr inner join TransactionReason as r on tr.TransactionReasonCodeID = r.TransactionReasonCodeID and cd.ChargeDetailID = r.ChargeDetailID)", 'dbtype': 'mssql', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run denailqueryeditHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "2399", "mean": "252.21", "stddev": "458.82", "min": "5.0", "max": "5736.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"feature_label": "DateOfEntry", "transformation_label": "Extract Date"}], "feature": "DateOfEntry", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "DateOfEntry"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PrimaryInsuranceBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "2399", "mean": "0.08", "stddev": "3.3", "min": "-28.91", "max": "121.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PrimaryInsuranceBal"}, {"transformationsData": [{"feature_label": "CPTModifier", "transformation_label": "String Indexer"}], "feature": "CPTModifier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2399", "mean": "31.05", "stddev": "13.18", "min": "00", "max": "XU", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTModifier"}, {"transformationsData": [{"feature_label": "PracticeCategory", "transformation_label": "String Indexer"}], "feature": "PracticeCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2399", "mean": "", "stddev": "", "min": "Charles Lee Ginsburgh", "max": "TPMG", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PracticeCategory"}, {"transformationsData": [{"feature_label": "DateOfService", "transformation_label": "Extract Date"}], "feature": "DateOfService", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "DateOfService"}, {"transformationsData": [{"feature_label": "CPTCategory", "transformation_label": "String Indexer"}], "feature": "CPTCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2399", "mean": "119527.27", "stddev": "512384.2", "min": "00731", "max": "Q4170", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCategory"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2399", "mean": "", "stddev": "", "min": "<Unspecified>", "max": "X-Ray", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"feature_label": "DiagnosisCodeID1", "std_scalar": {"mean_flag": "True", "std_flag": "False"}, "transformation_label": "Standard Scalar"}], "feature": "DiagnosisCodeID1", "transformation": [{"transformation": "Standard Scalar", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "min", "selected": "True", "stats": {"count": "2399", "mean": "41839.47", "stddev": "23100.1", "min": "13571", "max": "94869", "missing": "0"}, "updatedLabel": "DiagnosisCodeID1"}, {"transformationsData": [{"feature_label": "FacilityCategory", "transformation_label": "String Indexer"}], "feature": "FacilityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "2399", "mean": "", "stddev": "", "min": "After Hours Care Newport News", "max": "Yorktown Family Medicine", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityCategory"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2399", "mean": "0.2", "stddev": "0.4", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "status"}, {"feature": "DateOfEntry_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "2399", "mean": "16.13", "stddev": "8.23", "min": "1", "max": "30", "missing": "0"}, "updatedLabel": "DateOfEntry_dayofmonth"}, {"feature": "DateOfEntry_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "2399", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "DateOfEntry_month"}, {"feature": "DateOfEntry_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "2399", "mean": "2019.0", "stddev": "0.0", "min": "2019", "max": "2019", "missing": "0"}, "updatedLabel": "DateOfEntry_year"}, {"feature": "CPTModifier_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "2399", "mean": "2.95", "stddev": "4.75", "min": "0.0", "max": "41.0", "missing": "0"}, "updatedLabel": "CPTModifier_stringindexer..."}, {"feature": "PracticeCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "2399", "mean": "0.01", "stddev": "0.07", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "PracticeCategory_stringin..."}, {"feature": "DateOfService_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "2399", "mean": "15.07", "stddev": "8.31", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "DateOfService_dayofmonth"}, {"feature": "DateOfService_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "2399", "mean": "2.13", "stddev": "3.29", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "DateOfService_month"}, {"feature": "DateOfService_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "2399", "mean": "2018.89", "stddev": "0.31", "min": "2018", "max": "2019", "missing": "0"}, "updatedLabel": "DateOfService_year"}, {"feature": "CPTCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "2399", "mean": "30.93", "stddev": "52.47", "min": "0.0", "max": "266.0", "missing": "0"}, "updatedLabel": "CPTCategory_stringindexer..."}, {"feature": "DepartmentCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "2399", "mean": "1.6", "stddev": "2.57", "min": "0.0", "max": "25.0", "missing": "0"}, "updatedLabel": "DepartmentCategory_string..."}, {"feature": "FacilityCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "2399", "mean": "19.56", "stddev": "20.63", "min": "0.0", "max": "101.0", "missing": "0"}, "updatedLabel": "FacilityCategory_stringin..."}, {"feature": "DiagnosisCodeID1_standardscalar_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "selected": "True", "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "stats": {"count": "2399", "mean": "-0.0", "stddev": "23100.1", "min": "-28268.474781158824", "max": "53029.525218841176", "missing": "0"}, "updatedLabel": "DiagnosisCodeID1_standard..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run denailqueryeditHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["Amount", "PrimaryInsuranceBal", "CPTModifier_stringindexer", "PracticeCategory_stringindexer", "CPTCategory_stringindexer", "DepartmentCategory_stringindexer", "DiagnosisCodeID1_standardscalar", "FacilityCategory_stringindexer"], "status")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

